In [1]:
# Compare the modification dates in Databank and DDH for discrepancies
import numpy as np
import wbgapi as wb
import ddh
import sys
from datetime import datetime
# from tabulate import tabulate
# from IPython.display import HTML
import pandas as pd

ddh.load('datacatalog.worldbank.org')

In [2]:
headers = ['NID', 'DDH_TITLE', 'DDH_MODIFIED', 'DBANK_ID', 'DBANK_TITLE', 'DBANK_MODIFIED', 'DISCREPANCY']
# rows = []
df = pd.DataFrame(columns=headers)

for k,v in ddh.dataset.search(['field_wbddh_modified_date', 'field_ddh_harvest_sys_id'], {'field_ddh_harvest_src': 'Indicators API'}):
    database_id = v['field_ddh_harvest_sys_id']['und'][0]['value'] if v['field_ddh_harvest_sys_id'] else ''
    mod_date    = v['field_wbddh_modified_date']['und'][0]['value'] if v['field_wbddh_modified_date'] else ''
    mod_date = mod_date.split(' ')[0]
    
    if database_id:
        dbank = wb.source.get(database_id)
        delta = np.nan
        try:
            date1 = datetime.strptime(dbank['lastupdated'], '%Y-%m-%d')
            date2 = datetime.strptime(mod_date, '%Y-%m-%d')
            delta = (date2-date1).days
        except:
            pass
    else:
        dbank = {'name': '', 'lastupdated': ''}
        delta = np.nan

    row = [k, v['title'], mod_date, database_id, dbank['name'], dbank['lastupdated'], delta]
    # rows.append(row)
    df.loc[len(df)] = row

# HTML(tabulate(rows, tablefmt='html', headers=['NID','DDH_TITLE', 'DDH_MODIFIED', 'DBANK_ID', 'DBANK_TITLE', 'DBANK_MODIFIED', 'DISCREPANCY']))

This table shows DDH records whose modification dates don't match what's in Databank. To do this, it relies on the `field_ddh_harvest_sys_id` metadata field in DDH to link to the data API.

In [3]:
# Typical views:
#  df - entire report
#  df[df['DISCREPANCY']!=0] - just datasets without matching dates
#  df[df['DISCREPANCY']<0]  - just datasets where DDH is out of date
df[df['DISCREPANCY']!=0]

,NID,DDH_TITLE,DDH_MODIFIED,DBANK_ID,DBANK_TITLE,DBANK_MODIFIED,DISCREPANCY
0,94577,Global Economic Prospects,2020-01-08,27,Global Economic Prospects,2020-01-20,-12
1,94622,Quarterly Public Sector Debt,2019-12-31,20,Quarterly Public Sector Debt,2020-01-15,-15
8,392346,"Environment, Social and Governance Data",2019-12-19,75,"Environment, Social and Governance (ESG) Data",2020-01-14,-26
13,94554,Doing Business,2019-11-22,1,Doing Business,2019-10-23,30
37,140352,International Comparison Program 2011,2018-04-19,62,International Comparison Program (ICP) 2011,2017-05-09,345
41,94564,Enterprise Surveys,2017-01-30,13,Enterprise Surveys,2017-01-03,27
43,139420,Public Expenditure and Financial Accountabilit...,2016-02-26,,,,NaN
50,94576,Global Economic Monitor,,15,Global Economic Monitor,2019-09-13,NaN


**Footnotes:**

1. A negative `DISCREPANCY` means that the modification date in DDH is out of date and should be fixed.
2. It's unclear what a positive `DISCREPANCY` means; most likely that the dataset or its metadata were updated for reasons independent of Databank. For now, we're leaving these alone
3. *Global Economic Monitor* is updated every day and thus doesn't have a modification date in DDH, so this is normal
4. Databases in Databank that either aren't in DDH or don't have a correct `field_ddh_harvest_sys_id` field won't be included in the report; see next section

In [4]:
# now check which indicators are in the API but not in DDH
df2 = pd.DataFrame(columns=['DBANK_ID', 'DBANK_TITLE', 'DDH_NID', 'DDH_TITLE', 'STATUS'])
for db in wb.source.list():
    nid = None
    status = 'Missing'
    title = None
    for k,v in ddh.dataset.search({'field_ddh_harvest_src': 'Indicators API', 'field_ddh_harvest_sys_id': db['id']}):
        nid = k
        status = 'OK'
        title = v['title']
        break
    
    if status != 'OK':
        # try matching just on the system ID - maybe taxonomy is missing
        for k,v in ddh.dataset.search(['field_ddh_harvest_src'], {'field_ddh_harvest_sys_id': db['id']}):
            term = ddh.taxonomy.term('field_ddh_harvest_src', v)
            if term == None:
                # then likely they filled out the ID but not the source field
                nid = k
                title = v['title']
                status = 'Missing Harvest Src'
                break
    
    df2.loc[len(df2)] = [db['id'], db['name'], nid, title, status]

This report shows databases in the API that don't have matching records in DDH. This may indicate that DDH datasets need to be added or revised.

In [5]:
df2[df2['STATUS']!='OK']

,DBANK_ID,DBANK_TITLE,DDH_NID,DDH_TITLE,STATUS
21,30,Exporter Dynamics Database – Indicators at Cou...,None,None,Missing
28,37,LAC Equity Lab,None,None,Missing
49,67,PEFA 2011,None,None,Missing
50,68,PEFA 2016,None,None,Missing
52,70,Economic Fitness 2,None,None,Missing
54,72,PEFA_Test,None,None,Missing
55,73,Global Financial Inclusion and Consumer Protec...,None,None,Missing


**Footnotes:**
    
1. There is a single "Economic Fitness" dataset in DDH that references 2 Databank databases. This dataset should link (via `harvest_sys_id`) to the most recent database. The other(s) will show as missing, which is to be expected.
2. The PEFA databases are all classified as "official use" so they probably shouldn't be showing in the public API
